## ewf-ext-02-03-07 - NDVI anomalies of growing season per parcel 

NDVI anomalies of growing season per parcel

---

### <a name="service">Service definition

In [1]:
service = dict([('title', 'NDVI anomalies of growing season per parcel'),
                ('abstract', 'NDVI anomalies of growing season per parcel'),
                ('id', 'ewf-ext-02-03-07')])

### <a name="parameter">Parameter Definition 

In [2]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON ((-8.864205 38.88616500000001, -8.864205 38.986165, -8.964205000000002 38.986165, -8.964205000000002 38.88616500000001, -8.864205 38.88616500000001))'),
                         ('title', 'WKT Polygon for the Region of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

In [3]:
nameOfRegion = dict([('id', 'nameOfRegion'),
                     ('value', 'P001'),
                     ('title', 'Name of Region'),
                     ('abstract', 'Name of the region of interest'),
                     ('minOccurs', '1')])

In [4]:
indexAndApiKeys = dict([('id', 'indexAndApiKeys'),
                        ('value', 'better-ext-02-03-02,AKCp5dKscBjS2xuYhwSmqgvk2GGQ1rpxgLH2NqT9sD915j16AXj8Vy1S8Jwr5cFPX3EvHQxAT,better-ext-02-03-06,AKCp5dKscBjYU72HVTdPAAJJxVZjxTvyRtnF6eEM9TZvLifvFCDttrBDDKwPVbXoQPsb7JGf8'),
                        ('title', 'index,apikey pairs'),
                        ('abstract', 'index,apikey pairs'),
                        ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the NDVI stats' identifiers

In [5]:
# 2015, 2015
#difNdvi
#input_identifiers = ('LE07_ndviStats_P001_2015005_2015365.xlsx', 'LTA_LE07_ndviStats_P001_2015_2015.xlsx')
input_identifiers = ('E58456AA001B0A163CE829A579EFE1F4D10D2580','D4127BC272572368F1A819BCBCC312DB734FFE19')

**Input references**

This is the MODIS stack catalogue references

In [6]:
#input_references = tuple(['https://catalog.terradue.com/modis/search?format=atom&uid={0}'.format(pid) for pid in input_identifiers])
input_references = ('https://catalog.terradue.com/better-ext-02-03-02/search?format=atom&uid=E58456AA001B0A163CE829A579EFE1F4D10D2580','https://catalog.terradue.com/better-ext-02-03-06/search?format=atom&uid=D4127BC272572368F1A819BCBCC312DB734FFE19')

**Data path**

This path defines where the data is staged-in. 

In [7]:
data_path = ""

**Aux folders**

In [8]:
output_folder = ''

#### Import Modules

In [9]:
import os
import shutil

import sys
import string
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.wkt import loads

import datetime

import pdb

import pandas as pd
import geopandas as gpd

import cioppy
ciop = cioppy.Cioppy()

#### Auxiliary vars

In [10]:
check_results = True

In [11]:
# get metadata from catalog
def get_input_metadata (input_refs, apikeys):
    
    # for each product get metadata
    Result_Prod = []
    
    for index,product_ref in enumerate(input_refs):
        
        for index in apikeys:
            if index in product_ref:
                cat_index = index
                cat_apikey = apikeys[index]
        
        # since the search is by identifier
        Result_Prod.append(ciop.search(end_point = product_ref,params =[],output_fields='self,identifier,startdate,enclosure,startdate,enddate,wkt,title',creds='{}:{}'.format(cat_index,cat_apikey))[0] )
    

    input_metadata = gpd.GeoDataFrame.from_dict(Result_Prod)

    input_metadata['startdate'] = pd.to_datetime(input_metadata['startdate'])
    input_metadata['enddate'] = pd.to_datetime(input_metadata['enddate'])
    
    return input_metadata


def get_formatted_date(datetime_str):
    date = datetime.datetime.strftime(datetime_str, '%Y-%m-%dT%H:%M:%SZ')
 
    return date


def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    

    first_date_str = get_formatted_date(first_date)
    last_date_str = get_formatted_date(last_date)

    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date_str, last_date_str))
        file.write('geometry=%s' % (region_of_interest))

#### Workflow

Load data (agg)

In [12]:
# organize indexes and apikeys in a python dictionary
indexAndApiKeys_splited = indexAndApiKeys['value'].split(',')
apikeys = {}
for idx,ele in enumerate(indexAndApiKeys_splited):
    
    if (idx % 2 == 0):
        print(ele)
        apikeys[ele] = indexAndApiKeys_splited[idx+1]
        

# get input data from catalog
input_metadata = get_input_metadata (input_references, apikeys)



input_metadata_LTA = input_metadata[input_metadata['title'].str.find('LTA') != -1]
input_metadata_Agg = input_metadata[input_metadata['title'].str.find('LTA') == -1]

better-ext-02-03-02
better-ext-02-03-06


In [13]:
# get file paths
filepath_agg = os.path.join(data_path, input_metadata_Agg['enclosure'].iloc[0].split('/')[-1])
filepath_LTA = os.path.join(data_path, input_metadata_LTA['enclosure'].iloc[0].split('/')[-1])

In [14]:

# load data into a python dictionary
# key -> variable name
# content -> list of pandas dataframe, one per season (TS)
data_agg = {}

var_names = ['start_growing_season', 'end_growing_season', 'dif_ndvi', 'cumulative_ndvi', 'peak_ndvi']

f = filepath_agg
for var in var_names:

    df = pd.read_excel (f, sheet_name=var)
    #print (df)
    
    # remove useless column
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
    
    data_agg[var] = df

Load data (LTA)

In [15]:
# load data into a python dictionary
# key -> variable name
# content -> list of pandas dataframe, one per season (TS)
data_LTA = {}

var_names = ['start_growing_season', 'end_growing_season', 'dif_ndvi', 'cumulative_ndvi', 'peak_ndvi']

f = filepath_LTA
for var in var_names:

    df = pd.read_excel (f, sheet_name=var)
    #print (df)
    
    # remove useless column
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
    
    data_LTA[var] = df

Compute Amomalies

In [16]:
# new python dictionary to store LTAs
anom_data = {}

anom_data = data_agg.copy()

# to each var computes mean
for var in var_names:

    #anom_data[var][var] = data_LTA[var][var].div(data_agg[var][var])
    anom_data[var][var] = data_agg[var][var].div(data_LTA[var][var])


#### write output

In [17]:
name_parts = filepath_agg.split('/')[-1].split('.')[0].split('_')

#Anomaly_LE07_difNdvi_P001_2015005_2015020_LTA_2015-2015

mission = name_parts[0]
prod = name_parts[1]
aoi_name = name_parts[2]
start_date = name_parts[3]
end_date = name_parts[4]

name_parts = filepath_LTA.split('/')[-1].split('.')[0].split('_')

start_year_LTA = name_parts[4]
end_year_LTA = name_parts[5]

#start_date = str(anom_data['start_growing_season']['start_date'][0].year)
#end_date = str(anom_data['start_growing_season']['end_date'][0].year)

excel_output_name = '_'.join(['Anomaly', mission, prod, aoi_name, start_date, end_date, 'LTA', start_year_LTA, end_year_LTA]) + '.xlsx'
    
excel_output_name = os.path.join(output_folder, excel_output_name)

print(excel_output_name)


with pd.ExcelWriter(excel_output_name) as writer:  # doctest: +SKIP
    
    for key in anom_data:
    
        anom_data[key].to_excel(writer, sheet_name=key)


Anomaly_LE07_ndviStats_P001_2016005_2016365_LTA_2015_2017.xlsx


#### write properties file

In [18]:
write_properties_file(excel_output_name, input_metadata_Agg['startdate'].iloc[0], input_metadata_Agg['enddate'].iloc[0], regionOfInterest['value'])